# Eat Safe, Love

## Notebook Set Up

In [ ]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our database
print(db.list_collection_names())

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
# Find the establishments with a hygiene score of 20
query_hygene = establishments.find({'scores.Hygiene': 20})

# Use count_documents to display the number of documents in the result
count1 = establishments.count_documents({'scores.Hygiene': 20})
# Display the first document in the results using pprint
first_restaurant_hygiene_20 = establishments.find_one({'scores.Hygiene': 20})
pprint(f'number of restaurants with Hygene score of 20: {count1}')
pprint(first_restaurant_hygiene_20)

In [ ]:
# Convert the result to a Pandas DataFrame
df_Hygene = pd.DataFrame(list(query_hygene))

# Display the number of rows in the DataFrame
num_rows = df_Hygene.shape[0]
print("Number of rows in the DataFrame:", num_rows)

# Display the first 10 rows of the DataFrame
df_Hygene.head()

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query_london_rating = establishments.find({
    'AddressLine4':'London',
    'RatingValue' : {'$gte':4}
})

# Use count_documents to display the number of documents in the result
count_london_rating = establishments.count_documents({
    'AddressLine4':'London',
    'RatingValue':{'$gte':4}
})
print("Number of documents with AddressLine4 as 'London' and RatingValue >= 4:", count_london_rating)

# Display the first document in the results using pprint
first_london_rating = establishments.find_one({
    'AddressLine4':'London',
    'RatingValue':{'$gte':4}
})
print(f'The first document with London as local authority and Rating Value equal or greater than 4 is: {first_london_rating}')

In [ ]:
# Convert the result to a Pandas DataFrame
df_london_rating = pd.DataFrame(list(query_london_rating))

# Display the number of rows in the DataFrame
print("Number of rows in the DataFrame:", df_london_rating.shape[0])
# Display the first 10 rows of the DataFrame
df_london_rating.head(10)

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.0783519967076
longitude = 1.18590330311705

# Define the search range of ±0.01 degrees
latitude_min = target_latitude - degree_search
latitude_max = target_latitude + degree_search
longitude_min = target_longitude - degree_search
longitude_max = target_longitude + degree_search


query = establishments.find({
    'geocode.latitude': {'$gte': latitude_min, '$lte': latitude_max},
    'geocode.longitude': {'$gte': longitude_min, '$lte': longitude_max},
    'RatingValue': 5
})
sort = query.sort('scores.Hygiene', 1)  # Sort by Hygiene score in ascending order



# Print the results
matching_count = establishments.count_documents({
    'geocode.latitude': {'$gte': latitude_min, '$lte': latitude_max},
    'geocode.longitude': {'$gte': longitude_min, '$lte': longitude_max},
    'RatingValue': 5
})

print("Number of matching documents:", matching_count)

In [ ]:
# Convert result to Pandas DataFrame
df_results = pd.DataFrame(list(query))

# Display the sorted DataFrame
df_results.head()

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
pipeline = [
    # Step 1: Match documents with a Hygiene score of 0
    {'$match': {'scores.Hygiene': 0}},
    
    # Step 2: Group by AddressLine4 and count the number of occurrences
    {'$group': {
        '_id': '$AddressLine4',  # Group by AddressLine4
        'count': {'$sum': 1}     # Count the number of documents in each group
    }},
    
    # Step 3: Sort the results by count in descending order
    {'$sort': {'count': -1}}
]

# Print the number of documents in the result
print("Number of documents with a Hygiene score of 0:", establishments.count_documents({'scores.Hygiene':0}))

# Print the first 10 results
first_10_matches = establishments.find(
    {'scores.Hygiene': 0},
    {'BusinessName': 1, '_id': 0}  # Project only the BusinessName field, exclude _id
).limit(10)

# Print each of the first 10 matches
for doc in first_10_matches:
    pprint(doc)

In [ ]:
# Convert the result to a Pandas DataFrame
results_pipeline = establishments.aggregate(pipeline)
df_results_pipeline = pd.DataFrame(list(results_pipeline))

# Display the number of rows in the DataFrame
print("Number of rows in the DataFrame:", df_results_pipeline.shape[0])

# Display the first 10 rows of the DataFrame
df_results_pipeline.head(10)